In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_file = '/Users/nevinaresh/Documents/CPET Research/BxBData.json'
outcomes_file = '/Users/nevinaresh/Documents/CPET Research/CPX database_outcomes_meds_10_22.xlsx'

In [3]:
#reads in the data from the json file
raw_data = pd.read_json(data_file).transpose()
raw_data.head(2)

,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,TestDuration,...,P_Diast,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain
1,[Margaret],[Raffin],[MRN-03907458],[Female],[12/26/1946],[157.5],[45.36],[10/11/2018],[4:11 PM],[15:30],...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
2,[MYEDOE],[MYEDOE],[MRN-08/STANFORD CONCERT HF 08-7009-08],[Male],[5/1/1955],[175.3],[107.95],[6/19/2017],[2:34 PM],[26:23],...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[REST, REST, REST, REST, REST, REST, REST, RES...","[22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22.8, 22....","[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[634, 634, 634, 634, 634, 634, 634, 634, 634, ...","[756, 756, 756, 756, 756, 756, 756, 756, 756, ...","[1.73, 1.77, 1.61, 1.55, 0.84, 2.28, 1.5, 1.87...","[1.74, 2.26, 2.2, 2.13, 2.16, 1.69, 2.98, 2.28...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,..."


In [4]:
#formats columns and removes unnecessary
from src.jsonParsing import preprocess

preprocessed_data = preprocess(raw_data)
preprocessed_data.head(2)

,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,testTime,...,Phase,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test
0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,4:11 PM,...,"[REST, REST, REST, REST, REST, REST, REST, RES...","[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00
1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,11:47 AM,...,"[NONE, REST, REST, REST, REST, REST, REST, RES...","[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00


In [5]:
#filters those rows that have an ID1 of the form 'MRN-'MRNID
from src.jsonParsing import validmrn

data_with_mrn = validmrn(preprocessed_data)
data_with_mrn.head(2)

/Users/nevinaresh/Documents/Stanford/CPET-AI/src/jsonParsing.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_mrn['MRN'] = pd.to_numeric(valid_mrn['ID1'].str.split('-').str[1], errors='coerce')


,level_0,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,...,Ambient_Temp,RH_Amb,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test,MRN
0,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22....","[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...","[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0
1,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22.5, 22....","[, , , , , , , , , , , , , , , , , , , , , , ,...","[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0


In [6]:
outcomes_data = pd.read_excel(outcomes_file, sheet_name="Outcomes_updated")
outcomes_data.head(2)

,ID,MRN,Test Date,MRN-test date,Death,death_date,death_yrs,pre_arrhythm_surg,pre_arrhythm_surg_date,pre_arrhythm_surg_yrs,...,post_stroke_yrs,post_valve_surgery_unspecified,post_valve_surgery_unspecified_date,post_valve_surgery_unspecified_yrs,post_ventricular_fib_flut,post_ventricular_fib_flut_date,post_ventricular_fib_flut_yrs,post_ventricular_tachycardia,post_ventricular_tachycardia_date,post_ventricular_tachycardia_yrs
0,1.0,4400,2016-04-29,4400-42489,0.0,NaT,NaN,NaN,NaT,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
1,2.0,84095,2012-09-07,84095-41159,0.0,NaT,NaN,0.0,NaT,NaN,...,NaN,0.0,NaT,NaN,0.0,NaT,NaN,1.0,2014-03-31,1.56


In [7]:
from src.outcomeParsing import preprocess

death_data = preprocess(outcomes_data)
death_data.head(2)

,MRN,Test Date,Death
0,4400,2016-04-29,0.0
1,84095,2012-09-07,0.0


In [9]:
#merge on date of test or if there data after that date use that???
outcomes_and_test = pd.merge(data_with_mrn, death_data, left_on=['MRN', 'testDate'], right_on=['MRN','Test Date'], how='left')

In [14]:
outcomes_and_test

,level_0,index,firstName,lastName,ID1,Gender,DOB,Height,Weight,testDate,...,Analyzer_Pressure,PB,Ti,Te,Dyspnea,Leg Pain,time_of_test,MRN,Test Date,Death
0,0,1,Margaret,Raffin,MRN-03907458,Female,12/26/1946,157.5,45.36,2018-10-11,...,"[639, 639, 639, 639, 640, 639, 639, 639, 640, ...","[755, 755, 755, 755, 755, 755, 755, 755, 755, ...","[1.07, 0.8, 1.05, 1.12, 1.23, 1.36, 1.2, 1.19,...","[2.21, 1.42, 1.65, 2.29, 1.94, 2.16, 2.18, 2.2...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-10-11 16:11:00,3907458.0,2018-10-11,0.0
1,1,3,Larry,GERSTON,MRN-27411321,Male,5/27/1946,175.3,83.5,2014-08-08,...,"[649, 649, 649, 649, 649, 649, 649, 649, 649, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[1.45, 1.54, 1.62, 1.59, 1.98, 1.29, 1.4, 1.49...","[2.42, 2.51, 2.36, 2.06, 6, 2.63, 4.31, 3.15, ...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-08-08 11:47:00,27411321.0,2014-08-08,NaN
2,2,4,Adelita,MARFIA,MRN-10661601,Female,11/15/1951,152.4,73.5,2015-12-01,...,"[639, 639, 639, 639, 639, 639, 639, 639, 639, ...","[768, 768, 768, 768, 768, 768, 768, 768, 768, ...","[1.27, 1.12, 1.23, 1.95, 1.24, 1.56, 1.89, 1.5...","[2.74, 3.56, 1.73, 2.42, 2.6, 2.94, 3.2, 3.04,...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2015-12-01 09:04:00,10661601.0,2015-12-01,NaN
3,3,5,Gerald,FRASER,MRN-10269256,Male,7/10/1987,167.6,54,2016-01-11,...,"[635, 635, 636, 636, 636, 636, 636, 636, 636, ...","[769, 769, 769, 769, 769, 769, 769, 769, 769, ...","[2.22, 2.73, 0.69, 1.39, 1.4, 1.24, 1.38, 1.18...","[5.88, 3.4, 1.69, 2.1, 2.19, 1.89, 1.99, 1.91,...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2016-01-11 13:13:00,10269256.0,NaT,NaN
4,4,6,Yunjuan,Peng,MRN-40613937,Female,10/12/1976,165.1,62.6,2019-08-19,...,"[636, 636, 636, 636, 636, 636, 636, 636, 636, ...","[760, 760, 760, 760, 760, 760, 760, 760, 760, ...","[0.95, 1.32, 1.12, 1.3, 1.22, 1.79, 1.37, 1.26...","[4.93, 2.47, 2.17, 1.71, 2.28, 1.84, 2.19, 2.0...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2019-08-19 08:52:00,40613937.0,2019-08-19,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,4215,4217,Kara,NETTE,MRN-60452968,Female,7/1/1992,157.5,47.63,2018-05-30,...,"[628, 628, 628, 628, 628, 628, 628, 628, 628, ...","[759, 759, 759, 759, 759, 759, 759, 759, 759, ...","[1.03, 1.27, 1.01, 1.15, 1.44, 0.84, 1.2, 1.12...","[1.32, 1.56, 0.84, 0.9, 1.53, 1.64, 1.49, 1.34...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2018-05-30 10:08:00,60452968.0,2018-05-30,0.0
3880,4216,4218,Sophia,Zamoyski,MRN-74146523,Female,11/20/1999,185.4,70.31,2019-03-18,...,"[641, 641, 641, 641, 641, 641, 641, 641, 641, ...","[761, 761, 761, 761, 761, 761, 761, 761, 761, ...","[5.21, 0.68, 0.79, 1.66, 1.27, 0.97, 0.88, 0.9...","[1.79, 0.79, 1.64, 2.68, 1.63, 2.4, 1.21, 1.66...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2019-03-18 10:06:00,74146523.0,2019-03-18,0.0
3881,4217,4219,Adrian,HINOJOSA,MRN-23642655,Male,9/10/1993,177.8,57.2,2014-07-31,...,"[648, 648, 648, 648, 649, 649, 649, 649, 648, ...","[761, 761, 761, 761, 761, 761, 761, 761, 761, ...","[1.12, 1.14, 1.16, 1.55, 1.16, 1.29, 1.22, 1.5...","[1.75, 5.35, 2.36, 3.17, 2.66, 2.68, 2.56, 2.8...","[LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, LEVEL_6, ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",2014-07-31 14:00:00,23642655.0,NaT,NaN
3882,4218,4220,Jennifer,WOODS,MRN-25462326,Female,5/26/1969,167.6,69.9,2016-02-16,...,"[625, 625, 625, 625, 625, 625, 625, 625, 625, ...","[761, 761, 761, 761, 761, 761, 761, 761, 761, ...","[1.21, 1.15, 1.43, 1.64, 1.36, 1.29, 1.45, 1.1...","[1.97, 2.54, 2.92, 2.56, 2.03, 